In [4]:
import os
import sys

In [5]:
sys.path.insert(0, os.path.abspath(r".."))

In [6]:
from autopager.autopager import get_shared_autopager
from autopager.preprocessing import generate_page_component

In [9]:
from settings import RABBIT_ACCOUNT, RABBIT_PASSWORD, RABBIT_HOST, RABBIT_PORT, EVENT_QUEUE

In [7]:
import pika
import json
class Queue():
    def __init__(self, host, port, username, password):
        self.connection = pika.BlockingConnection(pika.ConnectionParameters(host,port=port,credentials=pika.PlainCredentials(username=username,password=password), heartbeat=0))
        self.channel = self.connection.channel()
        self.channel.basic_qos(prefetch_count=1)
    def DeclareQueue(self, queuename):
        self.channel.queue_declare(queue = queuename)
    def AddToQueue(self, queuename, data):
        self.channel.basic_publish(exchange = '', routing_key = queuename, body = json.dumps(data), properties = pika.BasicProperties(delivery_mode=2))
    def GetFromQueue(self, queuename, callback, auto_ack):
        self.channel.basic_consume(queue = queuename, on_message_callback = callback,auto_ack= auto_ack)
        self.channel.start_consuming()
    def Close(self):
        self.connection.close()

In [17]:
test_queue = Queue(RABBIT_HOST, RABBIT_PORT, RABBIT_ACCOUNT, RABBIT_PASSWORD)

In [20]:
test_queue.AddToQueue("testQueue", "https://nba.udn.com/nba/cate/6754/-1/newest/3")

In [12]:
def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)
    print(" [x] Done")
    ch.basic_ack(delivery_tag=method.delivery_tag)

In [13]:
test_queue.GetFromQueue("testQueue", callback, False)

KeyboardInterrupt: 

In [14]:
test_queue.Close()